In [9]:
import numpy as np
from pyfinite import ffield

In [10]:
#Final Matrices as obtained from Cryptanalysis.ipynb
At= [[84, 0, 0, 0, 0, 0, 0, 0], [115, 70, 0, 0, 0, 0, 0, 0], [21, 17, 43, 0, 0, 0, 0, 0], [124, 21, 7, 12, 0, 0, 0, 0], [99, 32, 1, 104, 112, 0, 0, 0], [18, 48, 18, 41, 101, 11, 0, 0], [12, 120, 15, 104, 3, 93, 27, 0], [71, 15, 84, 27, 10, 73, 28, 38]]
E = [19, 107, 37, 79, 89, 47, 24, 20]
A=[]
for i in range(len(At[0])):
    t=[]
    for j in range(len(At)):
        t.append(At[i][j])
    A.append(t) 
A       


[[84, 0, 0, 0, 0, 0, 0, 0],
 [115, 70, 0, 0, 0, 0, 0, 0],
 [21, 17, 43, 0, 0, 0, 0, 0],
 [124, 21, 7, 12, 0, 0, 0, 0],
 [99, 32, 1, 104, 112, 0, 0, 0],
 [18, 48, 18, 41, 101, 11, 0, 0],
 [12, 120, 15, 104, 3, 93, 27, 0],
 [71, 15, 84, 27, 10, 73, 28, 38]]

In [11]:
B_Size = 8
F = ffield.FField(7, gen=0x83, useLUT=-1)
A = np.array((A))
augmentedA = np.zeros((B_Size, B_Size*2), dtype = int)
inverseA = np.zeros((B_Size, B_Size), dtype = int)

Einverse = np.zeros((128, 128), dtype = int)

exponents = []
i=0
while i<128:
    exponents.append([1]) 
    i=i+1   
i=0
while i<128:
    for exponent in range(1,127):
        temp = exponents[i][exponent-1]
        result = F.Multiply(temp, i)
        exponents[i]+=[result]
    i=i+1    

j=0
while j<128:
    for exponent in range(1,127):
        Einverse[exponent][exponents[j][exponent]] = j
    j=j+1    

inverses = [1]
i=1
while i<128:
    inverses+=[F.Inverse(i)]
    assert F.Multiply(i, inverses[i]) == 1
    i=i+1

i=0
while i<B_Size:
    for j in range(0,B_Size):
        augmentedA[i][j] = A[i][j]
    augmentedA[i][i+j+1] = 1
    i=i+1
j=0
while j<B_Size:
    assert np.any(augmentedA[j:,j] != 0) # assert pivot row exists: A is invertible
    pivot_row = np.where(augmentedA[j:,j] != 0)[0][0] + j
    augmentedA[[j, pivot_row]] = augmentedA[[pivot_row, j]]
    # make pivot k 1
    mul_fact = inverses[augmentedA[j][j]]
    k=0
    while k<B_Size*2:
        augmentedA[j][k] = F.Multiply(augmentedA[j][k], mul_fact)
        k=k+1
    for i in range(0,B_Size):
        if i!=j and augmentedA[i][j] != 0:
            mult_fact = augmentedA[i][j]
            k=0
            while k<B_Size*2:
                temp = F.Multiply(augmentedA[j][k], mult_fact)
                augmentedA[i][k] = F.Add(temp, augmentedA[i][k])
                k=k+1
    j=j+1            
i=0
while i<B_Size:
    for j in range(0,B_Size):
        inverseA[i][j] = augmentedA[i][B_Size+j]
    i=i+1    
print("A inverse matrix: \n{}".format(inverseA))

A inverse matrix: 
[[ 67   0   0   0   0   0   0   0]
 [ 80  77   0   0   0   0   0   0]
 [  4  70  96   0   0   0   0   0]
 [ 82 105  56  94   0   0   0   0]
 [105  53   3 120  19   0   0   0]
 [  4  62  21  43  69  74   0   0]
 [115  72  86 104  53   4  14   0]
 [ 47  18 113  31  24 125  31  56]]


In [12]:
password = "gsjokqisgjmrkjjjmrltlfkkmikklpko" #Encrypted password
B_Size = 16
num_blocks = int(len(password) / B_Size) # 2 blocks

def Einverse_calculator(block, E):
    transformed = []
    j=0
    while j<8:
        transformed+=[Einverse[E[j]][block[j]]]
        j=j+1
    return transformed

def Ainverse_calculator(block, A):
    transformed = []
    row_num=0
    while row_num<8:
        elem_sum = 0
        for col_num in range(0,8):
            elem = F.Multiply(A[row_num][col_num], block[col_num])
            elem_sum = F.Add(elem, elem_sum)
        transformed+=[elem_sum]
        row_num=row_num+1
    return transformed

decrypted_password = ""
i=0
while i<2: # Decipher both the blocks of the password
    elements = password[B_Size*i:B_Size*(i+1)]
    currentBlock = []
    j=0
    while j<8:
        currentBlock+=[(ord(elements[2*j]) - ord('f'))*16 + (ord(elements[2*j+1]) - ord('f'))]
        j=j+1
    
    temp1 = Ainverse_calculator(Einverse_calculator(currentBlock, E), inverseA)
    EA = Einverse_calculator(temp1, E)
    
    temp2 = Ainverse_calculator(EA, inverseA)
    EAEAE = Einverse_calculator(temp2, E)
    
    print("Inverse EAEAE values for block:",str(i)+": ",EAEAE)
    for ch in EAEAE:
        decrypted_password += chr(ch)
    i=i+1    
print("The password is",decrypted_password)
    

Inverse EAEAE values for block: 0:  [115, 108, 110, 117, 97, 116, 113, 113]
Inverse EAEAE values for block: 1:  [102, 111, 48, 48, 48, 48, 48, 48]
The password is slnuatqqfo000000
